# Bulk RNA-seq eQTL analysis, external_bed

This notebook provide a master control on the XQTL workflow so it can automate the work before data preprocessing on multiple data collection as proposed.

This master control notebook is mainly to serve the 8 tissues snuc_bulk_expression analysis, but should be functional on all analysis where expression data an

Input:
    A recipe file,each row is a data collection and with the following column:
    
    Theme
        name of dataset, must be different, each uni_study analysis will be performed in a folder named after each, meta analysis will be performed in a folder named as {study1}_{study2}
            
        The column name must contain the # and be the first column
    
    genotype_file
        {Path to a whole genome genotype file}
    
    molecular_pheno
        {Path to file}
        
    covariate_file
        {Path to file}
    
    ### note: Only data collection from the same Populations and conditions will me merged to perform Fix effect meta analysis
    
Output:
    ...

In [ ]:

nohup sos run ~/GIT/xqtl-pipeline/code/complete_analysis/eQTL_analysis_externalBed.ipynb factor --recipe /mnt/mfs/statgen/snuc_pseudo_bulk//data/recipe_8tissue_new \
    --annotation_gtf /mnt/mfs/statgen/snuc_pseudo_bulk/data/reference_data/genes.reformatted.gene.gtf \
    --sample_participant_lookup /mnt/mfs/statgen/snuc_pseudo_bulk/data/reference_data/sampleSheetAfterQC.txt -s build &

nohup sos dryrun ~/GIT/xqtl-pipeline/code/complete_analysis/eQTL_analysis_externalBed.ipynb TensorQTL \
    --recipe /mnt/mfs/statgen/snuc_pseudo_bulk//data/recipe_8tissue_new     \
    --genotype_list /mnt/mfs/statgen/snuc_pseudo_bulk/data/genotype_qced/plink_files_list.txt      \
    --annotation_gtf /mnt/mfs/statgen/snuc_pseudo_bulk/data/reference_data/genes.reformatted.gene.gtf     \
    --sample_participant_lookup /mnt/mfs/statgen/snuc_pseudo_bulk/data/reference_data/sampleSheetAfterQC.txt -s build &





In [ ]:
/mnt/mfs/ctcn/team/masashi/snuc-eqtl/v20211109.celltypes/*/covariates-20211118.tsv

## Example for running the workflow
This will run the workflow from via several submission and save the output to nohup.out

In [ ]:
sos dryrun ~/GIT/xqtl-pipeline/code/complete_analysis/eQTL_analysis_externalBed.ipynb TensorQTL \
    --recipe /mnt/mfs/statgen/snuc_pseudo_bulk//data/recipe_8tissue_new     \
    --genotype_list /mnt/mfs/statgen/snuc_pseudo_bulk/data/genotype_qced/plink_files_list.txt      \
    --annotation_gtf /mnt/mfs/statgen/snuc_pseudo_bulk/data/reference_data/genes.reformatted.gene.gtf     \
    --sample_participant_lookup /mnt/mfs/statgen/snuc_pseudo_bulk/data/reference_data/sampleSheetAfterQC.txt   > Eight_tissue_scripts &

In [ ]:
[global]
## The aforementioned input recipe
parameter: recipe = path
## Overall wd, the file structure of analysis is wd/[steps]/[sub_dir for each steps]
parameter:  cwd = path(".")
## Diretory to the excutable
parameter: exe_dir = path("~/GIT/xqtl-pipeline/")
parameter: container = '/mnt/mfs/statgen/containers/twas_latest.sif'
parameter: container_base_bioinfo = '/mnt/mfs/statgen/containers/bioinfo.sif'
parameter: container_apex = '/mnt/mfs/statgen/containers/apex.sif'
parameter: container_PEER = '/mnt/mfs/statgen/containers/PEER.sif'
parameter: container_TensorQTL = '/mnt/mfs/statgen/containers//TensorQTL.sif'
parameter: container_rnaquant = '/mnt/mfs/statgen/containers/rna_quantification.sif'
parameter: container_flashpca = '/mnt/mfs/statgen/containers/flashpcaR.sif'
parameter: annotation_gtf = path
parameter: sample_participant_lookup = path
parameter: phenotype_id_type = "gene_name" 
parameter: yml = path("csg.yml")
import pandas as pd
input_inv = pd.read_csv(recipe, sep = "\t").to_dict("records")
import os

## Molecular Phenotype Calling

## Data Preprocessing
### Molecular Phenotype Processing

In [ ]:
#[Normalization]
#import os
#input: for_each = "input_inv"
#skip_if( os.path.exists(_input_inv["molecular_pheno"]))
#output: f'{cwd:a}/data_preprocessing/normalization/{name}.mol_phe.bed.gz'
#bash:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
#    sos run $[exe_dir]/data_preprocessing/phenotype/GWAS_QC.ipynb output \
#        --counts_gct $[_input_inv["genecount_table"]] \
#        --tpm_gct $[_input_inv["geneTpm_table"]] \
#        --sample_participant_lookup $[_input_inv["sample_index"]] \
#        --vcf_chr_list $[_input_inv["vcf_chr_list"]] \
#        --container $[container_gtex] \
#        --name $[_input_inv["Theme"]] \
#        --wd $[wd:a]/data_preprocessing/normalization/ \
#        --container $[container_base_bioinfo] \
#        -J 200 -q csg -c  $[yml] &

In [ ]:
[annotation]
## Must be ran with internet connection
import os
input: for_each = "input_inv"
output: f'{cwd:a}/data_preprocessing/{_input_inv["Theme"]}/phenotype_data/{path(_input_inv["molecular_pheno"]):bn}.bed.gz'
report:  expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
  sos run $[exe_dir]/pipeline/gene_annotation.ipynb annotate_coord \
    --cwd $[_output:d] \
    --phenoFile $[_input_inv["molecular_pheno"]] \
    --annotation-gtf $[annotation_gtf] \
    --sample-participant-lookup $[sample_participant_lookup] \
    --container $[container_rnaquant] \
    --phenotype-id-type $[phenotype_id_type] -J 1 -c $[yml] -q csg 

In [ ]:
[region_list_generation]
input: output_from("annotation"), group_with = "input_inv"
output: pheno_mod = f'{cwd:a}/data_preprocessing/{_input_inv["Theme"]}/phenotype_data/{_input:bnn}.region_list'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/pipeline/gene_annotation.ipynb region_list_generation \
        --cwd $[_output:d]  \
        --phenoFile $[_input]\
        --annotation-gtf $[annotation_gtf] \
        --sample-participant-lookup $[sample_participant_lookup] \
        --container $[container_rnaquant] \
        --phenotype-id-type $[phenotype_id_type] -J 1 -c $[yml] -q csg 

In [ ]:
[phenotype_partition_by_chrom]
input: output_from("annotation"),output_from("region_list_generation"), group_with = "input_inv"
output: per_chrom_pheno_list = f'{cwd:a}/data_preprocessing/{_input_inv["Theme"]}/phenotype_data/{_input[0]:bn}.processed_phenotype.per_chrom.recipe'        
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/pipeline/phenotype_formatting.ipynb partition_by_chrom \
        --cwd $[_output:d]  \
        --phenoFile $[_input[0]:a] \
        --region-list $[_input[1]:a] \
        --container $[container_rnaquant] \
        -J 30 -c $[yml] -q csg \
        --mem 4G 

### Genotype Processing
Since genotype is shared among the eight tissue, the QC of whole genome file is not needed. Only pca needed to be run again.

In [ ]:
[sample_match]
input: for_each = "input_inv"
output: f'{cwd:a}/data_preprocessing/{_input_inv["Theme"]}/{sample_participant_lookup:bn}.filtered.txt',
        geno = f'{cwd:a}/data_preprocessing/{_input_inv["Theme"]}/{sample_participant_lookup:bn}.filtered_geno.txt'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/pipeline/sample_matcher.ipynb filtered_sample_list \
        --cwd $[_output[0]:d]  \
        --phenoFile $[_input_inv["molecular_pheno"]]  \
        --genoFile $[path(_input_inv["genotype_file"]):n].fam  \
        --sample-participant-lookup $[sample_participant_lookup] \
        --container $[container_rnaquant] \
        --translated_phenoFile -J 1 -c $[yml] -q csg 

In [ ]:
[king]
input:output_from("sample_match")["geno"], group_with = "input_inv"
output: related = f'{cwd:a}/data_preprocessing/{_input_inv["Theme"]}/genotype_data/{path(_input_inv["genotype_file"]):bn}.{_input_inv["Theme"]}.related.bed',
        unrelated = f'{cwd:a}/data_preprocessing/{_input_inv["Theme"]}/genotype_data/{path(_input_inv["genotype_file"]):bn}.{_input_inv["Theme"]}.unrelated.bed'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
 sos run $[exe_dir]/pipeline/GWAS_QC.ipynb king \
    --cwd $[_output[0]:d] \
    --genoFile $[_input_inv["genotype_file"]] \
    --name $[_input_inv["Theme"]] \
    --keep-samples $[_input] \
    --container $[container_base_bioinfo]  -J 1 -c $[yml] -q csg \
    --walltime 48h -s force 

In [ ]:
[unrelated_QC]
input: output_from("king")["unrelated"]
output: unrelated_bed = f'{_input:n}.filtered.prune.bed', 
        prune = f'{_input:n}.filtered.prune.in'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
 sos run $[exe_dir]/pipeline/GWAS_QC.ipynb qc \
    --cwd $[_output[0]:d] \
    --genoFile $[_input] \
    --exclude-variants /mnt/mfs/statgen/snuc_pseudo_bulk/Ast/genotype/dupe_snp_to_exclude \
    --maf-filter 0.05 \
    --container $[container_base_bioinfo] -J 1 -c $[yml] -q csg \
    --mem 40G  

In [ ]:
[related_QC]
input: output_from("king")["related"],output_from("unrelated_QC")["prune"]
output: f'{_input[0]:n}.filtered.extracted.bed'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
 sos run $[exe_dir]/pipeline/GWAS_QC.ipynb qc_no_prune \
    --cwd $[_output[0]:d] \
    --genoFile $[_input[0]] \
    --exclude-variants /mnt/mfs/statgen/snuc_pseudo_bulk/Ast/genotype/dupe_snp_to_exclude \
    --maf-filter 0 \
    --geno-filter 0 \
    --mind-filter 0.1 \
    --hwe-filter 0 \
    --keep-variants $[_input[1]]  \
    --container $[container_base_bioinfo] -J 1 -c $[yml] -q csg \
    --mem 40G  

## Factor analysis

In [ ]:
[pca]
input: output_from("unrelated_QC")["unrelated_bed"],group_with = "input_inv"
output: f'{cwd}/data_preprocessing/{_input_inv["Theme"]}/pca/{_input:bn}.pca.rds',
        f'{cwd}/data_preprocessing/{_input_inv["Theme"]}/pca/{_input:bn}.pca.scree.txt'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
     sos run $[exe_dir]/pipeline/PCA.ipynb flashpca \
        --cwd $[_output:d] \
        --genoFile $[_input] \
        --container $[container_flashpca] -J 1 -c $[yml] -q csg  

In [ ]:
[projected_sample]
# The percentage 
parameter: PVE_treshold = 0.7
input: output_from("related_QC"),output_from("pca"), group_with = "input_inv"
output: f'{cwd}/data_preprocessing/{_input_inv["Theme"]}/pca/{_input[0]:bn}.pca.projected.rds'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/pipeline/PCA.ipynb project_samples \
            --cwd $[_output:d] \
            --genoFile $[_input[0]] \
            --pca-model  $[_input[1]] \
            --maha-k `awk '$3 < $[PVE_treshold]' $[_input[2]] | tail -1 | cut -f 1  ` \
            --k `awk '$3 < $[PVE_treshold]' $[_input[2]] | tail -1 | cut -f 1 ` \
            --container $[container_flashpca] -J 1 -c $[yml] -q csg 

In [ ]:
[merge_pca_covariate]
input: output_from("projected_sample"),group_with = "input_inv"
output: f'{cwd}/data_preprocessing/{_input_inv["Theme"]}/covariates/{path(_input_inv["covariate_file"]):bn}.pca.cov.gz'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/pipeline/covariate_formatting.ipynb merge_pca_covariate \
    --cwd $[_output:d] \
    --pcaFile $[_input:a] \
    --covFile  $[path(_input_inv["covariate_file"])] \
    --nCov 6  \
    --container $[container_base_bioinfo] -J 1 -c $[yml] -q csg 

In [ ]:
[factor]
input: output_from("merge_pca_covariate"),output_from("annotation"),group_with = "input_inv"
output: f'{cwd}/data_preprocessing/{_input_inv["Theme"]}/covariates/{_input[0]:bnn}.BiCV.cov.gz'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
     sos run $[exe_dir]/pipeline/BiCV_factor.ipynb BiCV \
        --cwd $[_output:d] \
        --phenoFile $[_input[1]:a]  \
        --covFile  $[_input[0]:a]  \
        --container $[container_apex]  -J 1 -c $[yml] -q csg \
        --walltime 24h \
        --numThreads 8 \
        --iteration 1000 \
        --N 60  

## Association scan

In [ ]:
[TensorQTL]
parameter: genotype_list = path
input: genotype_list, output_from("phenotype_partition_by_chrom"),output_from("factor"),group_with = "input_inv"
output: f'{cwd:a}/Association_scan/{_input_inv["Theme"]}/TensorQTL_recipe.tsv'
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/pipeline/TensorQTL.ipynb TensorQTL_cis \
        --genotype_file_list $[_input[0]]   \
        --molecular_pheno_list $[_input[1]] \
        --covariate $[_input[2]]    \
        --cwd $[_output:d]  \
        --container $[container_TensorQTL] -J 30 -c csg.yml -q csg  

In [ ]:
## RDS creation

In [ ]:
[rds_creation]
parameter: genotype_list = path
input: genotype_list, output_from("annotation"),output_from("factor"),output_from("region_list_generation"),group_with = "input_inv"
report:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/pipeline/per_gene_data_merger.ipynb data_merger \
        --genoFile $[_input[0]]   \
        --phenoFile $[_input[1]] \
        --covFile $[_input[2]]    \
        --cwd $[_output:d] \
        --region-list $[_input[3]] \
        --container $[container_TensorQTL] -J 30 -c csg.yml -q csg  